In [1]:
from dash import Dash, html, dcc, Input, Output, dash_table
import pandas as pd
import plotly.express as px

In [2]:
df = pd.read_csv("data/parte_1.csv").drop("Unnamed: 0", axis=1)
# df2 = pd.read_csv("data/parte_2.csv").drop("Unnamed: 0", axis=1)

# df = pd.concat([df, df2])
df.date = pd.to_datetime(df.date)

C:\Users\jurko\AppData\Local\Temp\ipykernel_6880\3364123337.py:1: DtypeWarning: Columns (7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/parte_1.csv").drop("Unnamed: 0", axis=1)


In [3]:
app = Dash(__name__)

app.title = "Trabajo Final"

In [4]:

tabs_styles = None
tab_style = None
tab_selected_style = None
tabs_styles = {
    'height': '44px'
}
tab_style = {
    'borderBottom': '1px solid #d6d6d6',
    'padding': '6px',
    'fontWeight': 'bold'
}
tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#119DFF',
    'color': 'white',
    'padding': '6px'
}

In [5]:
grouped_dates = df.set_index("date").groupby(pd.Grouper(freq='M')).sum().index
first_month = grouped_dates[0].strftime("%m-%Y")
last_month = grouped_dates[-1].strftime("%m-%Y")

In [6]:
fig = px.bar(x = ["tiendas", "productos", "estados", "meses"], y = [df["store_nbr"].nunique(), df["family"].nunique(), df["state"].nunique(), len(grouped_dates)], labels = {"x": "", "y": "Conteo"})

In [7]:
top_10 = df[["store_nbr","family", "sales"]].groupby("family").mean().sort_values("sales", ascending= False).head(10)
top_ten_products_graph = px.bar(top_10.reset_index(), x = "family", y = "sales")

ventas_por_tiendas = df.groupby("store_nbr").mean().sort_values("sales", ascending = False).reset_index()
top_ventas_por_tiendas_graph = px.bar(x = [str(x) for x in ventas_por_tiendas["store_nbr"]], y = ventas_por_tiendas["sales"], labels={"x":"store_nbr", "y":"sales"})

top_10_promocion = df[df["onpromotion"] == 1][["store_nbr","family", "sales"]].groupby("store_nbr").mean().sort_values("sales", ascending= False).head(10).reset_index()
top_10_promocion_graph = px.bar(x = [str(x) for x in top_10_promocion["store_nbr"]], y = top_10_promocion["sales"], labels={"x":"store_nbr", "y":"sales"})

In [8]:
@app.callback(
    Output(component_id='1_b',component_property='figure'),
    Input(component_id='b_dropdown',component_property='value'))
def update_graph(selected):
    if selected == "top_10_products":
        return top_ten_products_graph

    elif selected == "ventas_por_tiendas":
        return top_ventas_por_tiendas_graph

    elif selected == "top_10_promocion":
        return top_10_promocion_graph



### 1_C

In [9]:
dia_semana_ventas = df.groupby("day_of_week").mean()[["sales"]].reset_index()
dia_semana_ventas_graph = px.bar(dia_semana_ventas, x = "day_of_week", y ="sales")
dia_semana_ventas_graph = px.bar(dia_semana_ventas, x = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"], y ="sales", labels={"x":"day of week"})

ventas_semana_anno = df.groupby("week").mean()[["sales"]].reset_index()
ventas_semana_anno_graph = px.bar(ventas_semana_anno, x = "week", y = "sales")

ventas_mes_anno = df.groupby("month").mean()[["sales"]].reset_index()
ventas_mes_anno_graph = px.bar(ventas_mes_anno, x = "month", y = "sales")

In [10]:
@app.callback(
    Output(component_id='1_c',component_property='figure'),
    Input(component_id='c_dropdown',component_property='value'))
def update_graph(selected):
    if selected == "ventas_dia_semana":
        return dia_semana_ventas_graph
    
    elif selected == "ventas_por_semana_anno":
        return ventas_semana_anno_graph
    
    elif selected == "ventas_por_mes_anno":
        return ventas_mes_anno_graph

In [11]:
opciones_1_b = {"Ránking productos más vendidos": "top_10_products", 
                "Ventas por tiendas": "ventas_por_tiendas",
                "Ránking tiendas con más ventas en promoción": "top_10_promocion"}
opciones_1_c = {"Ventas por día de semana": "ventas_dia_semana",
                "Volumen de ventas medio por semana del año": "ventas_por_semana_anno",
                "Volumen de ventas medio por mes del año": "ventas_por_mes_anno"}
tab_1_content = html.Div([
    dcc.Graph(id = "1_a", figure = fig, style = {"width": "50%", 'display': 'inline-block',"height" : "30vh"}),
    html.Div([
        html.Table([html.Thead(html.Th("Meses de los que se tiene datos")),
                    html.Tbody([html.Tr([
                                    html.Td("Primer mes"),
                                    html.Td(first_month)]),
                                html.Tr([
                                    html.Td("Último mes"),
                                    html.Td(last_month)])])],
            style= {"margin": "auto", "position":"relative", "top": "50%", "transform": "translateY(-50%)", "border": "1px solid","border-radius": "5px", "padding":"5px 10px 5px 10px"})
    ], style = {"display": "inline-block", "width": "50%", "height" : "30vh"}),
    html.Div([
        dcc.Dropdown(id='b_dropdown', ## dropdown menu
                    options=[{'label':k,'value':v} for k, v in opciones_1_b.items()],
                    value='top_10_products'),
        dcc.Graph(id = "1_b"),

    ], style  = {"width": "60%",'display': 'inline-block', "height" : "40vh"}),
    html.Div([

        dcc.Dropdown(id='c_dropdown', ## dropdown menu
                    options=[{'label':k,'value':v} for k, v in opciones_1_c.items()],
                    value='ventas_dia_semana'),
        dcc.Graph(id = "1_c")
    ], style  = {"width": "40%",'display': 'inline-block', "height" : "40vh"})
])

In [12]:
tab_1 = dcc.Tab(label = "Visualización Global",
                value = "tab_1",
                style = tab_style,
                selected_style=tab_selected_style,
                children = tab_1_content)

In [13]:
selected_store_nbr_global = 1
tab_2_content = html.Div(children = [
    dcc.Dropdown(id='store_dropdown', ## dropdown menu
                options=[{'label':store_nbr,'value':store_nbr} for store_nbr in sorted(df["store_nbr"].unique())],
                value=1), 
    html.Div(id = "tab_2_div")
])

In [14]:
@app.callback(
    Output(component_id='tab_2_div',component_property='children'),
    Input(component_id='store_dropdown',component_property='value'))
def update_graph(selected):
    store_df = df[df["store_nbr"] == selected]


    total_sales_per_year = store_df.groupby("year").sum()[["sales"]].reset_index()
    products_promotion = pd.DataFrame(data = {"Sold without promotion": [len(store_df[store_df["onpromotion"] == 0])], "Sold on promotion": [len(store_df[store_df["onpromotion"] == 1])]})
    products_promotion["Conteo"] = ""
    products_promotion = products_promotion.set_index("Conteo")


    year_graph = px.line(total_sales_per_year, x = "year", y = "sales")
    year_graph.update_xaxes(type = "category")
    
    to_return = html.Div([
        html.Div([
            dcc.Graph(id = "graph_2_a",figure = year_graph),
        ], style = {"width": "70%", "display": "inline-block"}),
        html.Div([

            dcc.Graph(id = "graph_2_b", figure = px.bar(products_promotion ,barmode='stack', text_auto= True))
        ], style = {"width": "30%", "display": "inline-block"})
    ], style = {"display":"block", "width":"100%", "position":"absolute", "top":"50%", "transform": "translateY(-50%)"})


    return to_return

In [15]:

tab_2 = dcc.Tab(label = "Información por tienda",
                value = "tab_2",
                style = tab_style,
                selected_style=tab_selected_style,
                children = tab_2_content)

In [16]:
tab_3_content = html.Div([
    dcc.Dropdown(id='state_dropdown', ## dropdown menu
                options=[{'label':state,'value':state} for state in sorted(df["state"].unique())],
                value="Pichincha"), 
    html.Div(id = "tab_3_div")
])

In [17]:
state_df = df[df["state"] == "Pichincha"]
state_year = state_df.groupby("year").sum()[["transactions"]].reset_index()
state_year.year = pd.to_datetime(state_year.year, format = "%Y").dt.year

In [18]:
@app.callback(
    Output(component_id='tab_3_div',component_property='children'),
    Input(component_id='state_dropdown',component_property='value'))
def update_div(selected):
    state_df = df[df["state"] == selected]
    state_year = state_df.groupby("year").sum()[["transactions"]].reset_index()
    state_year.year = pd.to_datetime(state_year.year, format = "%Y").dt.year

    state_sales = state_df.groupby("store_nbr").sum().sort_values("sales", ascending = True)[["sales"]].reset_index()
    state_sales["store_nbr"] = state_sales["store_nbr"].apply(str)

    most_sold_product = state_df.groupby("family").sum()[["sales"]].sort_values("sales", ascending=False).reset_index()

    fig = px.line(state_year, x = "year", y = "transactions")
    fig.update_xaxes(type='category')

    to_return = html.Div([
        dcc.Graph(figure = fig, style = {"height": "40vh"}),
        html.Div([
            dcc.Graph(figure = px.bar(state_sales, x = "sales", y = "store_nbr", orientation="h"), config={'responsive': True}, style = {"height": "100%"}),
        ],style = {"grid-row": "1 / 4", "grid-column": "2"}),
        dcc.Dropdown(id = "dropdown_3_c", value = "todas las tiendas", options = [{"label": i, "value": i} for i in ["todas las tiendas", *sorted(state_df.store_nbr.unique())]]),
        dcc.Graph(id = "graph_3_c", style = {"height": "45vh"}),
    ], style = {"display":"grid", "grid-template-columns": "60% 40%"})

    return to_return

In [19]:
@app.callback(
    Output(component_id='graph_3_c',component_property='figure'),
    Input(component_id='dropdown_3_c',component_property='value'))
def update_graph(selected):
    
    if selected != "todas las tiendas":
        most_sold_product = state_df[state_df["store_nbr"] == selected].groupby("family").sum()[["sales"]].sort_values("sales", ascending=False).reset_index().head(10)
    else:
        most_sold_product = state_df.groupby("family").sum()[["sales"]].sort_values("sales", ascending=False).reset_index().head(10)

    fig = px.bar(most_sold_product, x = "family", y = "sales")

    return fig

In [20]:
tab_3 = dcc.Tab(label = "Información por Estado",
                value = "tab_3",
                style = tab_style,
                selected_style=tab_selected_style,
                children = tab_3_content)

In [21]:
tab_4_content = html.Div()

In [22]:
tab_4 = dcc.Tab(label = "Información Extra",
                value = "tab_4",
                style = tab_style,
                selected_style=tab_selected_style,
                children = tab_4_content)

In [23]:
app.layout = html.Div([
    dcc.Tabs(
        id = "tabs", value = "tab_1",
        children = [tab_1, tab_2, tab_3, tab_4],
        
    ),
], style = {"font-family": "arial"})

In [24]:
app.run_server(jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/
